# Vertex AI - Object detection using AutoML (using APIs)

### Summary

In this notebook, we'll use Vertex AI to train customer object detection model using image dataset. After training, we will create an endpoint deploy the model to an endpoint and deploy the model to that endpoint. Then we will use the deploymed model for online prediction. 

### Prerequisites

 - Images dataset with object to identify

### Items Covered in this notebook

 - Setup resources
 - Image dataset creation
 - Import images in the dataset and create labels
 - Train custom AutoML models for object tdetection
 - Evaluate the trained model
 - Create and endpoint for deploy model 
 - Perform online prediction on new data
 - Cleanup

#### Setup resources

In [11]:
# Load required packages to run the notebook

import json
import requests
from typing import List, Union
from IPython.display import Image

from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import trainingjob

In [4]:
# Setup variables for notebooks

LOCATION = "us-central1" 
PROJECT_ID = "nishitp-daml"
DATASET_ID = "4715062101670887424"
EXPORT_DIRECTORY = "gs://ds-ml-demos/packages-image-metadata/"
ANNOTATION_SET_ID = "package-classification_iod"
FILTER = f"labels.aiplatform.googleapis.com/annotation_set_name={ANNOTATION_SET_ID}"

In [5]:
# Creating helper functions for API calls

def manage_package_dataset_metadata(method, endpoint, data, auth_token):
    data = str(data)
    headers = {'Content-type': 'application/json', "Authorization": f"Bearer {auth_token}"}
    
    if method == "GET":
        resp = requests.get(endpoint, headers=headers)
    if method == "POST":
        resp = requests.post(endpoint, data=data, headers=headers)
    if method == "DELETE":
        resp = requests.delete(endpoint, headers=headers)
    
    return(resp.json())

##### Let's create image dataset in the Vertex AI and import all images into our dataset

From the GCP console left task list pane, find and click on `Vertex AI` and then click on `Dataset` option to create a new dataset

![](files/images/dm-01-gcp-console-vertex-ai-image-dataset.png)

Once you are in the dataset `Create dataset` page, provide a name for dataset. We name our dataset `package-detection-ds` for this demo. Next, select `IMAGE` as dataset type and click on `Image object detection` option. Let's keep region and other setting as default for now. Next, click on `CREATE` button to create a new dataset resoure in Vertex AI.

![](files/images/dm-02-gcp-console-vertex-ai-image-dataset-creation.png)

When the dataset is created, you will click on the dataset and import the images for labeling and training the model. You have following options to import images from within Verte AI dataset:

- Upload from your local machine (limit 500 per upload)
- Upload import file from your local machine (this file will contains the path to images and other attributes)
- Select & import images from Cloud storage bucket (provide gcs location of the file. either jsonl or csv format)

For this demo, we will use the file that we already have in the cloud stoage bucket and import. To do this, click on `IMPORT` pane and select `Select import files form Cloud Storage` option and provide path to the file and click on `BROWSE` button to select the gcs storage bucket and file. Lastly, click on `CONTINUE` button to import the images. 

![](files/images/dm-03-gcp-console-vertex-ai-image-dataset-import.png)

Once, all the images have been imported (based on dataset size it could take a while to load everything), you can browse it from the `BROWSE` pane of the dataset. You'll be able to create new labels for images and create boundaries around the object that you are trying to detect in the images. For example in this demo, we are trying to identify what kind of packages was delivered at the door. The labels are as follows:

- small_packages
- medium_packages
- big_packages
- envelopes

![](files/images/dm-03-gcp-console-vertex-ai-image-dataset-browse.png)

After labeling all the images in our dataset, we are now ready to train the model. To do this, we will be using Vertex AI APIs. First, will create a function that will take custom inputs and starts the `Vertex AI training` resource.

Below are the function input description:

- `project`: your gcp project id
- `display_name`: vertex ai training resournce name in the console
- `dataset_id`: dataset id from previous step (check console->vertex ai->datasets->ID)
- `model_display_name`: trained model name shown in console
- `location`: location of your image dataset
- `api_endpoint`: gcp vertex ai api endpoint

In [3]:
# Training automl function

def train_package_size_detection(
    project: str,
    display_name: str,
    dataset_id: str,
    model_display_name: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests. This only needs to be created once. 
    aip_client = aiplatform.gapic.PipelineServiceClient(client_options=client_options)
    
    training_inputs = trainingjob.definition.AutoMlImageObjectDetectionInputs(
        model_type="CLOUD_HIGH_ACCURACY_1", # goal is to train a model with high accuracy as opposed to low latency serving here
        budget_milli_node_hours=20000,      # this is the minimum numbee of node hours requires to train object detection model
        disable_early_stopping=False,
    ).to_value()

    training_pipeline = {
        "display_name": display_name,
        "training_task_definition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml",
        "training_task_inputs": training_inputs,
        "input_data_config": {"dataset_id": dataset_id},
        "model_to_upload": {"display_name": model_display_name},
    }
    parent = f"projects/{project}/locations/{location}"
    response = aip_client.create_training_pipeline(
        parent=parent, training_pipeline=training_pipeline
    )
    print("response:", response)


Let's train the model now